# Test de création d'isochrones avec de nouvelles formes

On se base sur l'API de Navitia pour obtenir les points qui permettent de composer les isochrones

In [1]:
# Add module to path
import os
import sys

add_path = os.getcwd().replace('experiments', '')
sys.path.append(add_path)

In [2]:
from dotenv import load_dotenv
import docopt
import json
from pathlib import Path
from csv_to_json import csv_to_json
from isos_and_intersections import GetIso

param_csv = os.path.join(add_path, "params/test_journeys.csv")
param_json = os.path.join(add_path, "params/test_journeys.json")
places_cache = os.path.join(add_path, "params/places_cache.json")

#Parameters
env_path = Path(add_path) / '.env'
load_dotenv(dotenv_path=env_path)
TOKEN = os.getenv("NAVITIA_TOKEN")

columns_with_array_of_str = [
    "colors_iso",
    "addresses",
    "excluded_modes",
    "durations"
    ]

#JSON INPUT
json_file = csv_to_json(param_csv, param_json, ";", columns_with_array_of_str)

gdf_global = GetIso(param_json, places_cache, api="navitia", token=TOKEN).get_all_isos()

In [9]:
gdf_global["details"]

,direction,geometry,id,mode,name,duration,from_place
0,NaN,POINT (2.347884 48.858303),https://api.navitia.io/v1/coverage/fr-idf/jour...,NaN,Châtelet,1200,2.3467922;48.8621487
1,Mairie de Montrouge (Montrouge),"LINESTRING (2.346495 48.862015, 2.347884 48.85...",https://api.navitia.io/v1/coverage/fr-idf/jour...,Métro,Porte de Clignancourt - Mairie de Montrouge,1200,2.3467922;48.8621487
2,NR,"LINESTRING (2.347884 48.858303, 2.3480483993 4...",https://api.navitia.io/v1/coverage/fr-idf/jour...,NR,NR,1200,2.3467922;48.8621487
0,NaN,POINT (2.35264 48.866049),https://api.navitia.io/v1/coverage/fr-idf/jour...,NaN,Réaumur-Sébastopol,1200,2.3467922;48.8621487
1,Porte de Clignancourt (Paris),"LINESTRING (2.346495 48.862015, 2.349002 48.86...",https://api.navitia.io/v1/coverage/fr-idf/jour...,Métro,Porte de Clignancourt - Mairie de Montrouge,1200,2.3467922;48.8621487
2,NR,"LINESTRING (2.35264 48.866049, 2.3525355889 48...",https://api.navitia.io/v1/coverage/fr-idf/jour...,NR,NR,1200,2.3467922;48.8621487
0,NaN,POINT (2.346222 48.855688),https://api.navitia.io/v1/coverage/fr-idf/jour...,NaN,Cité,1200,2.3467922;48.8621487
1,Mairie de Montrouge (Montrouge),"LINESTRING (2.346495 48.862015, 2.347884 48.85...",https://api.navitia.io/v1/coverage/fr-idf/jour...,Métro,Porte de Clignancourt - Mairie de Montrouge,1200,2.3467922;48.8621487
2,NR,"LINESTRING (2.346222 48.855688, 2.3461702391 4...",https://api.navitia.io/v1/coverage/fr-idf/jour...,NR,NR,1200,2.3467922;48.8621487
0,NaN,POINT (2.329927 48.872157),https://api.navitia.io/v1/coverage/fr-idf/jour...,NaN,Auber,1200,2.3467922;48.8621487


In [8]:
gdf_global["journeys_details"].to_csv(os.path.join(add_path, "params/journeys_details.csv"))

AttributeError: 'NoneType' object has no attribute 'to_csv'